# Lab-Data-Manipulation - Group By and Joins

In [1]:
# setup from last exercise

import pandas as pd

ufo = pd.read_csv('data/ufo.csv', sep = ';')
ufo = ufo.rename(columns = {'encounter_length': 'encounter_seconds'})
ufo_vars = ufo.loc[:, ['year', 'month', 'state', 'country', 'ufo_shape', 'encounter_seconds']]
ufo_us = ufo_vars.loc[ufo_vars['country'] == 'us', :]

The second part of this lab consists of grouping and merging results.

# Grouping up the results. 

## Let's calculate the average of the encounter for each country.

We should now group the results by the country column to see what is the mean encounter_seconds for each country. Do this using the `groupby` method of your dataframe `ufo_vars`. What is the average of the encounter for the us? And for Canada?

Remember that after grouping by a column, you have to specify a `aggregating function`. If you don't do that, the results of the groupby will only be a `groupby` pandas object. For this case, we want the aggregating function to be the `mean` function and then the results will appear for us.

Also remember that **if you don't** specify the `as_index=False` argument, the variables you use to group are going to become your new indexes.

Expected output:

>
>|    | country   |    year |   month |   encounter_seconds |
|---:|:----------|--------:|--------:|--------------------:|
|  0 | au        | 2002.69 | 6.12825 |             3806.47 |
|  1 | ca        | 2003.69 | 6.86167 |            28859.43 |
|  2 | de        | 2001.54 | 6.24762 |            24255.98 |
|  3 | gb        | 2003.65 | 6.5622  |            66061.32 |
|  4 | us        | 2004.09 | 6.8616  |             5800.19 |

In [2]:
ufo_vars.groupby('country', as_index = False).mean()

,country,year,month,encounter_seconds
0,au,2002.693309,6.128253,3806.469238
1,ca,2003.694333,6.861667,28859.437007
2,de,2001.542857,6.247619,24255.980952
3,gb,2003.648294,6.562205,66061.321207
4,us,2004.090057,6.861597,5800.192049


## Perform the same task, but instead of calculating the mean, count how many occurrances for each country.

For this case, the aggregating function should be the `count` function. Try to understand the results for each column.

Expected output:

>
>|    | country   |   year |   month |   state |   ufo_shape |   encounter_seconds |
|---:|:----------|-------:|--------:|--------:|------------:|--------------------:|
|  0 | au        |    538 |     538 |      10 |         527 |                 538 |
|  1 | ca        |   3000 |    3000 |    2985 |        2955 |                3000 |
|  2 | de        |    105 |     105 |       0 |         103 |                 105 |
|  3 | gb        |   1905 |    1905 |      12 |        1855 |                1905 |
|  4 | us        |  65114 |   65114 |   65114 |       63561 |               65112 |

In [3]:
ufo_vars.groupby('country', as_index = False).count()

,country,year,month,state,ufo_shape,encounter_seconds
0,au,538,538,10,527,538
1,ca,3000,3000,2985,2955,3000
2,de,105,105,0,103,105
3,gb,1905,1905,12,1855,1905
4,us,65114,65114,65114,63561,65112


## Perform the same task, but instead of calculating the mean, use the `.describe()` aggregating function to see the effects.

The describe aggregating function will show you several important statistics for the grouped results, such as `mean`, `median`, `standard deviation`, `count`, `max`, `min`, and so on.

*Hint: If it starts to get difficult to see the results, you can tranpose the resulting dataframe by just putting a `.T` at the end.*

Expected output:

>
>|                   |   count |       mean |          std |      min |   25% |   50% |   75% |           max |
|:------------------|--------:|-----------:|-------------:|---------:|------:|------:|------:|--------------:|
| year              |   80332 | 2003.85    |     10.4268  | 1906     |  2001 |  2006 |  2011 | 2014          |
| month             |   80332 |    6.83491 |      3.23486 |    1     |     4 |     7 |     9 |   12          |
| encounter_seconds |   80329 | 9017.23    | 620228       |    0.001 |    30 |   180 |   600 |    97836000   |

In [4]:
ufo_vars.describe().T

,count,mean,std,min,25%,50%,75%,max
year,80332.0,2003.850085,10.426832,1906.000,2001.0,2006.0,2011.0,2014.0
month,80332.0,6.834910,3.234863,1.000,4.0,7.0,9.0,12.0
encounter_seconds,80329.0,9017.225634,620228.369318,0.001,30.0,180.0,600.0,97836000.0


## Now, let's get deeper in the analysis and group the results not only by country. But by `country` and `year`

### Check the values of the mean and count for the `encounter_seconds` variable for each year. Can you see some discrepancy?

*Hint*: If you want, you can use the `ufo_us` dataset just to see the results for the united states. You could also (in a hacky way) perform the filter right before the groupby operation if you wanted.

In [5]:
ufo_vars.groupby(['country', 'year']).agg(['mean', 'count'])['encounter_seconds']

mean  count
country year                     
au      1958   2700.000000      1
        1960    180.000000      1
        1967    300.000000      1
        1968    300.000000      1
        1972    403.333333      3
...                    ...    ...
us      2010   2271.987232   3548
        2011   2544.292555   4379
        2012  10640.644916   6320
        2013   1266.387888   6056
        2014    782.372979   1964

[282 rows x 2 columns]

# BONUS 2: Which months are the ones with the highest numbers of occurrences?

**Example of output:**

>
>|   month |   occurrences |
|--------:|--------------:|
|       7 |          9520 |
|       8 |          8636 |
|       6 |          8152 |
|       9 |          7589 |
|      10 |          7407 |
|      11 |          6739 |
|       1 |          5689 |
|      12 |          5665 |
|       4 |          5527 |
|       3 |          5450 |
|       5 |          5293 |
|       2 |          4665 |


In [6]:
ufo_vars.groupby('month').agg(occurrences = ('year', 'count')).sort_values(by = 'occurrences', ascending = False)

,occurrences
month,
7,9520
8,8636
6,8152
9,7589
10,7407
11,6739
1,5689
12,5665
4,5527


# BONUS 3: Finally, you gathered information about the UFO dataset. Using your last result, try to bring that information for your original dataset.

1. Store the results of your previous analysis (the mean value for the encounter_seconds for each year and each country) in a dataframe called `avg_results`. 

Expected output:
>
>|     | country   |   year |   encounter_seconds |
|----:|:----------|-------:|--------------------:|
|   0 | au        |   1958 |      2700           |
|   1 | au        |   1960 |       180           |
|   2 | au        |   1967 |       300           |
|   3 | au        |   1968 |       300           |
|   4 | au        |   1972 |       403.333       | 
|   ... | ...        |   ... |       ...       | 
| 277 | us        |   2010 |      2271.99        |
| 278 | us        |   2011 |      2544.29        |
| 279 | us        |   2012 |     10640.6         |
| 280 | us        |   2013 |      1266.39        |
| 281 | us        |   2014 |       782.373       |

In [7]:
avg_results = ufo_vars[['country', 'year', 'encounter_seconds']].groupby(['country', 'year']).mean().reset_index()
avg_results

,country,year,encounter_seconds
0,au,1958,2700.000000
1,au,1960,180.000000
2,au,1967,300.000000
3,au,1968,300.000000
4,au,1972,403.333333
...,...,...,...
277,us,2010,2271.987232
278,us,2011,2544.292555
279,us,2012,10640.644916
280,us,2013,1266.387888


2. Rename the column named `encounter_seconds` to `avg_encounter_seconds`.

Expected output:
>
>|     | country   |   year |   avg_encounter_seconds |
|----:|:----------|-------:|--------------------:|
|   0 | au        |   1958 |      2700           |
|   1 | au        |   1960 |       180           |
|   2 | au        |   1967 |       300           |
|   3 | au        |   1968 |       300           |
|   4 | au        |   1972 |       403.333       | 
|   ... | ...        |   ... |       ...       | 
| 277 | us        |   2010 |      2271.99        |
| 278 | us        |   2011 |      2544.29        |
| 279 | us        |   2012 |     10640.6         |
| 280 | us        |   2013 |      1266.39        |
| 281 | us        |   2014 |       782.373       |

In [8]:
avg_results = avg_results.rename({'encounter_seconds':'avg_encounter_seconds'}, axis = 1)
avg_results

,country,year,avg_encounter_seconds
0,au,1958,2700.000000
1,au,1960,180.000000
2,au,1967,300.000000
3,au,1968,300.000000
4,au,1972,403.333333
...,...,...,...
277,us,2010,2271.987232
278,us,2011,2544.292555
279,us,2012,10640.644916
280,us,2013,1266.387888


3. Use the pd.merge( ... ) function to bring that new collected information to your original dataset.
The pd.merge() function requires several arguments, let's understand the most important ones.

`left` is the dataframe you want to bring information **to** - the table on the left. In this case, this will be our original dataframe called `ufo`

`right` is the dataframe you want to bring information **from** - the table on the right. In this case, this will be our resulting dataframe `avg_results`.

`on` is the key you want to perform the merge. That is, if those values are **exactly equal** in both dataframes, then the information will be brought.

Put your results on a dataframe called `merged_ufo`

In [9]:
merged_ufo = pd.merge(left = ufo, right = avg_results, on = ['country', 'year'])
merged_ufo.head()

,Unnamed: 0,date,year,month,day,date_time,city_area,state,country,ufo_shape,encounter_seconds,described_encounter_length,description,date_documented,latitude,longitude,avg_encounter_seconds
0,0,1949-10-10,1949,10,10,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,523.333333
1,1706,1949-10-15,1949,10,15,10/15/1949 20:00,baltimore,md,us,cigar,180.0,2-3 minutes,It was around 8:PM and I was walking home&#44i...,3/4/2008,39.290278,-76.612500,523.333333
2,35615,1949-04-10,1949,4,10,4/10/1949 15:00,seattle,wa,us,disk,600.0,ten minutes,UFO Hovers over Seattle sky,4/27/2004,47.606389,-122.330833,523.333333
3,41489,1949-05-01,1949,5,1,5/1/1949 14:00,oak ridge (drive),wv,us,other,40.0,40seconds,log shape whooshing object observed near charl...,3/19/2002,38.075000,-81.109722,523.333333
4,42287,1949-05-15,1949,5,15,5/15/1949 06:30,grays,sc,us,light,300.0,2-5 min.,I am not sure of the exact date or time of thi...,10/31/2003,32.673611,-81.020556,523.333333


Check how many rows the final result has and try to explain it. Did the dataset get smaller? Bigger? Or the same? Can you explain why? 

*hint: If you've found something weird, try looking back at your original dataframe and find something interesting on it.*

In [10]:
merged_ufo.shape

(70662, 17)

## Store the results into a new csv file called `ufo_enriched.csv`. 

Don't forget to use `index=False`.

In [11]:
merged_ufo.to_csv('ufo_enriched.csv', index = False)